In [ ]:
# Local Interpretability: LIME and ICE Methods
# Implementation for LightGBM Model from Step2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Import interpretability libraries
import lime
import lime.lime_tabular
from pdpbox import pdp, info_plots
import shap

# Import model libraries
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries loaded successfully!")
print(f"LIME version: {lime.__version__}")
print(f"LightGBM version: {lgb.__version__}")


ModuleNotFoundError: No module named 'lime'

In [5]:
df = pd.read_csv("/Users/solalzana/Desktop/DSB - A2/Interpretability, Stability & Algorithmic Fairness/Algo_fairness-Group-Project/data/dataproject2025.csv")
df

,Unnamed: 0,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,...,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american,Predictions,Predicted probabilities
0,0,2013,0,39600.0,1379.0,21564.0,16.1,0.0,2.49,2 years,...,home_improvement,4136.0,16.1,B2,0,0.0,782,7.388592,0,0.053051
1,1,2013,0,55000.0,9570.0,16473.0,53.9,0.0,22.87,10+ years,...,debt_consolidation,36638.0,61.2,B2,0,0.0,481,9.745456,0,0.084507
2,2,2013,0,325000.0,53306.0,13901.0,67.1,0.0,18.55,5 years,...,debt_consolidation,29581.0,54.6,A3,0,0.0,945,7.542862,0,0.037206
3,3,2013,0,130000.0,36362.0,3567.0,93.0,0.0,13.03,10+ years,...,debt_consolidation,10805.0,67.0,B3,0,0.0,809,6.598132,0,0.061371
4,4,2013,1,73000.0,24161.0,4853.0,74.7,1.0,23.13,6 years,...,debt_consolidation,27003.0,82.8,D5,1,0.0,802,7.058900,1,0.345896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770684,973521,2016,0,60000.0,3096.0,5513.0,61.4,0.0,8.88,1 year,...,debt_consolidation,8787.0,61.4,B3,0,0.0,941,5.137052,0,0.098673
770685,973522,2016,0,86000.0,13225.0,9723.0,37.0,0.0,18.53,4 years,...,debt_consolidation,6518.0,31.4,C2,1,0.0,352,42.597439,0,0.208755
770686,973523,2016,1,35027.0,2948.0,313.0,92.2,0.0,28.79,9 years,...,home_improvement,9306.0,78.2,E1,1,0.0,307,3.388381,1,0.408135
770687,973524,2016,0,106756.0,34392.0,2708.0,76.5,3.0,10.53,10+ years,...,home_improvement,13492.0,74.2,A4,0,0.0,740,2.837343,0,0.028343


In [ ]:
# Train a LightGBM model (simplified version of Step2)
# If you have the trained model from Step2, you can load it instead

def train_simple_lightgbm_model(df, target_col='default', test_size=0.3):
    """
    Train a simplified LightGBM model for interpretability analysis
    """
    # Prepare features
    feature_cols = [col for col in df.columns if col not in [target_col]]
    X = df[feature_cols].copy()
    y = df[target_col].copy()
    
    # Simple preprocessing
    for col in X.columns:
        if X[col].dtype == 'object':
            # Handle categorical variables
            X[col] = pd.Categorical(X[col]).codes
        elif X[col].isnull().any():
            # Fill missing values
            X[col] = X[col].fillna(X[col].median())
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=SEED, stratify=y
    )
    
    # Train LightGBM model
    model = lgb.LGBMClassifier(
        objective='binary',
        metric='auc',
        n_estimators=100,
        learning_rate=0.1,
        num_leaves=31,
        random_state=SEED,
        verbose=-1
    )
    
    model.fit(X_train, y_train)
    
    # Evaluate
    train_pred = model.predict_proba(X_train)[:, 1]
    test_pred = model.predict_proba(X_test)[:, 1]
    
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)
    
    print(f"Training AUC: {train_auc:.4f}")
    print(f"Test AUC: {test_auc:.4f}")
    
    return model, X_train, X_test, y_train, y_test, feature_cols

# Train the model
model, X_train, X_test, y_train, y_test, feature_names = train_simple_lightgbm_model(df)

print(f"Model trained successfully!")
print(f"Features: {feature_names}")


# LIME Implementation

LIME (Local Interpretable Model-agnostic Explanations) explains individual predictions by approximating the model locally with a simpler, interpretable model.

## Key Features of LIME:
- **Model-agnostic**: Works with any machine learning model
- **Local explanations**: Explains individual predictions
- **Interpretable**: Provides human-understandable explanations
- **Feature importance**: Shows which features contribute most to the prediction


In [ ]:
# Initialize LIME Tabular Explainer
def create_lime_explainer(X_train, feature_names, class_names=['No Default', 'Default']):
    """
    Create a LIME tabular explainer for the dataset
    """
    explainer = lime.lime_tabular.LimeTabularExplainer(
        X_train.values,
        feature_names=feature_names,
        class_names=class_names,
        mode='classification',
        discretize_continuous=True,
        random_state=SEED
    )
    return explainer

# Create the explainer
lime_explainer = create_lime_explainer(X_train, feature_names)
print("LIME explainer created successfully!")

# Function to explain a single prediction
def explain_instance_with_lime(explainer, model, instance, num_features=10):
    """
    Explain a single instance using LIME
    """
    explanation = explainer.explain_instance(
        instance.values,
        model.predict_proba,
        num_features=num_features,
        top_labels=2
    )
    return explanation

# Select a few test instances to explain
instances_to_explain = [0, 1, 2, 10, 20]  # Indices of test instances
explanations = []

print("Generating LIME explanations for selected instances...")
for idx in instances_to_explain:
    exp = explain_instance_with_lime(lime_explainer, model, X_test.iloc[idx])
    explanations.append(exp)
    
    # Print prediction and actual label
    pred_proba = model.predict_proba(X_test.iloc[idx:idx+1])[0, 1]
    actual_label = y_test.iloc[idx]
    print(f"Instance {idx}: Predicted probability = {pred_proba:.3f}, Actual = {actual_label}")

print(f"Generated {len(explanations)} LIME explanations!")


In [ ]:
# Visualize LIME explanations
def plot_lime_explanation(explanation, instance_idx, save_fig=True):
    """
    Plot LIME explanation for a single instance
    """
    # Get the explanation for the positive class (Default)
    exp_list = explanation.as_list(label=1)
    
    # Extract feature names and weights
    features = [item[0] for item in exp_list]
    weights = [item[1] for item in exp_list]
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Color coding: positive weights (red) contribute to default, negative (green) contribute to no default
    colors = ['red' if w > 0 else 'green' for w in weights]
    
    # Create horizontal bar plot
    bars = ax.barh(range(len(features)), weights, color=colors, alpha=0.7)
    
    # Customize the plot
    ax.set_yticks(range(len(features)))
    ax.set_yticklabels(features)
    ax.set_xlabel('Feature Weight (Contribution to Default Prediction)')
    ax.set_title(f'LIME Explanation for Instance {instance_idx}')
    ax.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, weight) in enumerate(zip(bars, weights)):
        ax.text(weight + (0.01 if weight > 0 else -0.01), i, f'{weight:.3f}', 
                va='center', ha='left' if weight > 0 else 'right')
    
    # Add legend
    ax.legend(['Neutral', 'Increases Default Risk', 'Decreases Default Risk'], 
              loc='lower right')
    
    plt.tight_layout()
    
    if save_fig:
        plt.savefig(f'lime_explanation_instance_{instance_idx}.png', dpi=300, bbox_inches='tight')
    
    plt.show()
    
    return fig

# Plot explanations for the first few instances
print("Plotting LIME explanations...")
for i, (exp, idx) in enumerate(zip(explanations[:3], instances_to_explain[:3])):
    print(f"\n--- LIME Explanation for Instance {idx} ---")
    plot_lime_explanation(exp, idx)
    
    # Also print text summary
    print("Text summary:")
    for label_id in exp.available_labels():
        print(f"Class {label_id} ({['No Default', 'Default'][label_id]}):")
        exp_list = exp.as_list(label=label_id)
        for feature, weight in exp_list[:5]:  # Top 5 features
            print(f"  {feature}: {weight:.3f}")
    print("---")


# ICE (Individual Conditional Expectation) Implementation

ICE plots show how predictions change for individual instances as we vary a single feature while keeping all other features constant.

## Key Features of ICE:
- **Individual-level insights**: Shows how each instance responds to feature changes
- **Feature interaction detection**: Reveals non-linear relationships and interactions
- **Model behavior understanding**: Helps understand model's decision boundaries
- **Heterogeneity visualization**: Shows different responses across instances


In [ ]:
# ICE Plot Implementation using pdpbox
def create_ice_plot(model, X_data, feature_name, num_ice_lines=50, figsize=(12, 8)):
    """
    Create ICE (Individual Conditional Expectation) plot for a specific feature
    """
    try:
        # Create ICE plot using pdpbox
        ice_plot = pdp.pdp_isolate(
            model=model,
            dataset=X_data,
            model_features=list(X_data.columns),
            feature=feature_name,
            num_grid_points=20
        )
        
        # Create the plot
        fig, ax = plt.subplots(figsize=figsize)
        
        # Plot ICE lines for individual instances
        pdp.pdp_plot(
            ice_plot, 
            feature_name, 
            ice_lines=True,
            center=False,
            plot_lines=True,
            frac_to_plot=min(num_ice_lines/len(X_data), 1.0),
            ax=ax
        )
        
        ax.set_title(f'ICE Plot for {feature_name}')
        ax.set_ylabel('Predicted Probability of Default')
        ax.set_xlabel(feature_name)
        
        plt.tight_layout()
        plt.show()
        
        return ice_plot, fig
        
    except Exception as e:
        print(f"Error creating ICE plot with pdpbox: {e}")
        print("Creating manual ICE plot...")
        return create_manual_ice_plot(model, X_data, feature_name, num_ice_lines, figsize)

def create_manual_ice_plot(model, X_data, feature_name, num_ice_lines=50, figsize=(12, 8)):
    """
    Manual implementation of ICE plot
    """
    # Sample instances for ICE lines
    if len(X_data) > num_ice_lines:
        sample_indices = np.random.choice(len(X_data), num_ice_lines, replace=False)
        sample_data = X_data.iloc[sample_indices].copy()
    else:
        sample_data = X_data.copy()
    
    # Get feature range
    feature_min = X_data[feature_name].min()
    feature_max = X_data[feature_name].max()
    feature_values = np.linspace(feature_min, feature_max, 20)
    
    # Store predictions for each instance
    ice_predictions = []
    
    for idx, (_, instance) in enumerate(sample_data.iterrows()):
        instance_predictions = []
        for feature_val in feature_values:
            # Create modified instance
            modified_instance = instance.copy()
            modified_instance[feature_name] = feature_val
            
            # Get prediction
            pred = model.predict_proba([modified_instance.values])[0, 1]
            instance_predictions.append(pred)
        
        ice_predictions.append(instance_predictions)
    
    # Create the plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot individual ICE lines
    for i, predictions in enumerate(ice_predictions):
        ax.plot(feature_values, predictions, alpha=0.3, color='blue')
    
    # Plot average (PDP line)
    avg_predictions = np.mean(ice_predictions, axis=0)
    ax.plot(feature_values, avg_predictions, color='red', linewidth=3, label='Average (PDP)')
    
    ax.set_title(f'ICE Plot for {feature_name}')
    ax.set_xlabel(feature_name)
    ax.set_ylabel('Predicted Probability of Default')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {'feature_values': feature_values, 'ice_predictions': ice_predictions}, fig

# Create ICE plots for the most important features
print("Creating ICE plots for important features...")

# Get feature importance from the model
feature_importance = model.feature_importances_
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Feature Importance:")
print(importance_df.head(10))

# Select top features for ICE plots
top_features = importance_df.head(4)['feature'].tolist()
print(f"\nCreating ICE plots for top {len(top_features)} features: {top_features}")

ice_results = {}
for feature in top_features:
    print(f"\nCreating ICE plot for {feature}...")
    ice_data, fig = create_ice_plot(model, X_test.head(100), feature, num_ice_lines=30)
    ice_results[feature] = ice_data
